In [ ]:
import os
import tensorflow as tf
import numpy as np
import random
from PIL import Image
import albumentations as aug

In [ ]:
class dataloader(tf.keras.utils.Sequence):
    def __init__(self, data_location, batch_size, mode):
        self.batch_size = batch_size
        self.data_location = data_location
        self.tofloat = aug.ToFloat(max_value=255)
        imgs = []
        for i in os.listdir(data_location):
            if i.endswith(".png"):
                imgs.append(i)
        self.imgs = np.sort(np.array(imgs))
        
        if mode == "train":
            self.imgs = self.imgs[:-200]
        elif mode == "test":
            self.imgs = self.imgs[-100:]
        else:
            self.imgs = self.imgs[-200:-100]
        
        if (mode == "test"):
            self.transforms = aug.Compose([aug.RandomCrop(648, 648, False, 1.)])
        else:
            self.transforms = aug.Compose([aug.RandomCrop(648, 648, False, 1.),
                                           aug.HorizontalFlip(0.5),
                                           aug.ColorJitter(0.1, 0.1, 0.1, 0.1, False, 0.8)])
    
    def __len__(self):
        return len(self.imgs) // self.batch_size
    
    def on_epoch_end(self):
        random.shuffle(self.imgs)
    
    def __getitem__(self, idx):
        i = idx * self.batch_size
        batch_imgs = self.imgs[i : i + self.batch_size]
        batch_highres = np.zeros((self.batch_size, 648, 648, 3))
        batch_lowres = np.zeros((self.batch_size, 162, 162, 3))
    
        for i, img in enumerate(batch_imgs):
            highres_pil = Image.open(os.path.join(self.data_location, img))
            highres_np = np.array(highres_pil)
            highres_transform = self.transforms(image=highres_np)["image"]
            highres_pil = Image.fromarray(highres_transform)
            
            lowres_pil = highres_pil.resize((162, 162), resample=Image.BICUBIC)
            lowres_transform = np.array(lowres_pil)

            batch_highres[i] = self.tofloat(image=highres_transform)["image"]
            batch_lowres[i] = self.tofloat(image=lowres_transform)["image"]
        
        return (batch_lowres, batch_highres)
        
        

In [ ]:
def fsrcnn():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(162, 162, 3)))
    
    model.add(tf.keras.layers.Conv2D(filters=56, kernel_size=5, padding="same", kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(tf.keras.layers.PReLU(shared_axes=[1, 2]))
    
    model.add(tf.keras.layers.Conv2D(filters=12, kernel_size=1, padding="same", kernel_initializer=tf.keras.initializers.HeNormal()))
    model.add(tf.keras.layers.PReLU(shared_axes=[1, 2]))
    
    for i in range(4):
        model.add(tf.keras.layers.Conv2D(filters=12, kernel_size=3, padding="same", kernel_initializer=tf.keras.initializers.HeNormal()))
        model.add(tf.keras.layers.PReLU(shared_axes=[1, 2]))
    
    model.add(tf.keras.layers.Conv2D(filters=56, kernel_size=1, padding="same"))
    model.add(tf.keras.layers.PReLU(shared_axes=[1, 2]))
    
    model.add(tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=9, strides=4, padding="same", kernel_initializer=tf.keras.initializers.RandomNormal(mean=0, stddev=0.001)))
    
    return model

In [ ]:
train_data = dataloader("./data/", 30, "train")
val_data = dataloader("./data/", 20, "val")

In [ ]:
model = fsrcnn()

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss="mean_squared_error")

#callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.7, patience=15, min_lr=10e-6, verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=10e-6, patience=50, verbose=0, restore_best_weights=True)
save = tf.keras.callbacks.ModelCheckpoint(filepath="./weights/model_{epoch:03d}.h5", monitor="loss", save_best_only=True, save_weights_only=False, save_freq="epoch")

In [ ]:
model.fit(train_data, epochs=500, steps_per_epoch=20, callbacks=[reduce_lr, early_stopping, save], validation_data=val_data, validation_steps=4)

Epoch 1/500
20/20 [==============================] - 75s 3s/step - loss: 0.0613 - val_loss: 0.0336
Epoch 2/500
20/20 [==============================] - 69s 3s/step - loss: 0.0308 - val_loss: 0.0386
Epoch 3/500
20/20 [==============================] - 69s 3s/step - loss: 0.0279 - val_loss: 0.0315
Epoch 4/500
20/20 [==============================] - 69s 3s/step - loss: 0.0261 - val_loss: 0.0327
Epoch 5/500
20/20 [==============================] - 69s 3s/step - loss: 0.0221 - val_loss: 0.0237
Epoch 6/500
20/20 [==============================] - 69s 3s/step - loss: 0.0204 - val_loss: 0.0220
Epoch 7/500
20/20 [==============================] - 69s 3s/step - loss: 0.0184 - val_loss: 0.0228
Epoch 8/500
20/20 [==============================] - 69s 3s/step - loss: 0.0179 - val_loss: 0.0350
Epoch 9/500
20/20 [==============================] - 69s 3s/step - loss: 0.0188 - val_loss: 0.0184
Epoch 10/500
20/20 [==============================] - 69s 3s/step - loss: 0.0174 - val_loss: 0.0177
Epoch 11/

20/20 [==============================] - 69s 3s/step - loss: 0.0074 - val_loss: 0.0054
Epoch 84/500
20/20 [==============================] - 69s 3s/step - loss: 0.0074 - val_loss: 0.0049
Epoch 85/500
20/20 [==============================] - 69s 3s/step - loss: 0.0071 - val_loss: 0.0075
Epoch 86/500
20/20 [==============================] - 69s 3s/step - loss: 0.0077 - val_loss: 0.0065
Epoch 87/500
20/20 [==============================] - 69s 3s/step - loss: 0.0070 - val_loss: 0.0077
Epoch 88/500
20/20 [==============================] - 69s 3s/step - loss: 0.0081 - val_loss: 0.0061
Epoch 89/500
20/20 [==============================] - 69s 3s/step - loss: 0.0072 - val_loss: 0.0050
Epoch 90/500
20/20 [==============================] - 69s 3s/step - loss: 0.0067 - val_loss: 0.0095
Epoch 91/500
20/20 [==============================] - 69s 3s/step - loss: 0.0078 - val_loss: 0.0065
Epoch 92/500
20/20 [==============================] - 69s 3s/step - loss: 0.0072 - val_loss: 0.0107
Epoch 93/500


Epoch 163/500
20/20 [==============================] - 69s 3s/step - loss: 0.0047 - val_loss: 0.0047
Epoch 164/500
20/20 [==============================] - 69s 3s/step - loss: 0.0045 - val_loss: 0.0041
Epoch 165/500
20/20 [==============================] - 69s 3s/step - loss: 0.0049 - val_loss: 0.0041
Epoch 166/500
20/20 [==============================] - 69s 3s/step - loss: 0.0046 - val_loss: 0.0042
Epoch 167/500
20/20 [==============================] - 69s 3s/step - loss: 0.0048 - val_loss: 0.0043

Epoch 00167: ReduceLROnPlateau reducing learning rate to 0.00034300000406801696.
Epoch 168/500
20/20 [==============================] - 69s 3s/step - loss: 0.0043 - val_loss: 0.0043
Epoch 169/500
20/20 [==============================] - 69s 3s/step - loss: 0.0044 - val_loss: 0.0041
Epoch 170/500
20/20 [==============================] - 69s 3s/step - loss: 0.0043 - val_loss: 0.0044
Epoch 171/500
20/20 [==============================] - 69s 3s/step - loss: 0.0045 - val_loss: 0.0036
Epoch 172

20/20 [==============================] - 69s 3s/step - loss: 0.0040 - val_loss: 0.0035
Epoch 242/500
20/20 [==============================] - 69s 3s/step - loss: 0.0039 - val_loss: 0.0038
Epoch 243/500
20/20 [==============================] - 70s 3s/step - loss: 0.0041 - val_loss: 0.0033
Epoch 244/500
20/20 [==============================] - 69s 3s/step - loss: 0.0039 - val_loss: 0.0034
Epoch 245/500
20/20 [==============================] - 69s 3s/step - loss: 0.0039 - val_loss: 0.0035
Epoch 246/500
20/20 [==============================] - 69s 3s/step - loss: 0.0040 - val_loss: 0.0037

Epoch 00246: ReduceLROnPlateau reducing learning rate to 0.00011764899536501615.
Epoch 247/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0036
Epoch 248/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0039
Epoch 249/500
20/20 [==============================] - 69s 3s/step - loss: 0.0039 - val_loss: 0.0034
Epoch 250/500
20/20 [==

20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0038
Epoch 319/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0033
Epoch 320/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0036
Epoch 321/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0035
Epoch 322/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0033
Epoch 323/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0033
Epoch 324/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0038

Epoch 00324: ReduceLROnPlateau reducing learning rate to 1.977326610358432e-05.
Epoch 325/500
20/20 [==============================] - 69s 3s/step - loss: 0.0038 - val_loss: 0.0037
Epoch 326/500
20/20 [==============================] - 71s 4s/step - loss: 0.0035 - val_loss: 0.0036
Epoch 327/500
20/20 [===

In [ ]:
hist = _

In [ ]:
model.summary()

In [ ]:
hist